# 从nii文件生成DCM序列


In [1]:
import SimpleITK as sitk
import numpy as np
import os

## 1. nii文件基本信息

In [2]:
reader = sitk.ImageFileReader()
reader.SetFileName("E:\other program\DCMProcessor\dataset\ct.nii")
reader.ReadImageInformation()

# 获取nii文件MetaData
MetaData_keys = reader.GetMetaDataKeys()
print("MetaData:")
for key in MetaData_keys:
    print(f"{key}\t\t:{reader.GetMetaData(key)}")

MetaData:
ITK_FileNotes		:
aux_file		:
bitpix		:16
cal_max		:0
cal_min		:0
datatype		:4
descrip		:
dim[0]		:3
dim[1]		:512
dim[2]		:512
dim[3]		:851
dim[4]		:1
dim[5]		:1
dim[6]		:1
dim[7]		:1
dim_info		:0
intent_code		:0
intent_name		:
intent_p1		:0
intent_p2		:0
intent_p3		:0
nifti_type		:1
pixdim[0]		:0
pixdim[1]		:0.683
pixdim[2]		:0.683
pixdim[3]		:0.8
pixdim[4]		:0
pixdim[5]		:0
pixdim[6]		:0
pixdim[7]		:0
qform_code		:1
qform_code_name		:NIFTI_XFORM_SCANNER_ANAT
qoffset_x		:190.283
qoffset_y		:174.658
qoffset_z		:1274.5
quatern_b		:0
quatern_c		:0
quatern_d		:1
scl_inter		:0
scl_slope		:1
sform_code		:1
sform_code_name		:NIFTI_XFORM_SCANNER_ANAT
slice_code		:0
slice_duration		:0
slice_end		:0
slice_start		:0
srow_x		:-0.683 0 0 190.283
srow_y		:0 -0.683 0 174.658
srow_z		:0 0 0.8 1274.5
toffset		:0
vox_offset		:352
xyzt_units		:2


In [3]:
basic_info = (
    ("size\t", reader.GetSize()),
    ("dimension", reader.GetDimension()),
    ("origin\t", reader.GetOrigin()),
    ("spacing\t", reader.GetSpacing()),
    ("direction", reader.GetDirection()),
    ("dtype\t", sitk.GetPixelIDValueAsString(reader.GetPixelID())),
)
print("基本信息：")
for info in basic_info:
    print(f"{info[0]}\t:{info[1]}")

基本信息：
size		:(512, 512, 851)
dimension	:3
origin		:(-190.283203125, -174.658203125, 1274.5)
spacing		:(0.6830000281333923, 0.6830000281333923, 0.800000011920929)
direction	:(1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 0.0, 0.0, 1.0)
dtype		:16-bit signed integer


nii文件读取数据后得到的Image保有了nii的MetaData。

In [4]:
img = reader.Execute()
img.GetMetaDataKeys()

('ITK_FileNotes',
 'ITK_original_direction',
 'ITK_original_spacing',
 'aux_file',
 'bitpix',
 'cal_max',
 'cal_min',
 'datatype',
 'descrip',
 'dim[0]',
 'dim[1]',
 'dim[2]',
 'dim[3]',
 'dim[4]',
 'dim[5]',
 'dim[6]',
 'dim[7]',
 'dim_info',
 'intent_code',
 'intent_name',
 'intent_p1',
 'intent_p2',
 'intent_p3',
 'nifti_type',
 'pixdim[0]',
 'pixdim[1]',
 'pixdim[2]',
 'pixdim[3]',
 'pixdim[4]',
 'pixdim[5]',
 'pixdim[6]',
 'pixdim[7]',
 'qform_code',
 'qform_code_name',
 'qoffset_x',
 'qoffset_y',
 'qoffset_z',
 'quatern_b',
 'quatern_c',
 'quatern_d',
 'scl_inter',
 'scl_slope',
 'sform_code',
 'sform_code_name',
 'slice_code',
 'slice_duration',
 'slice_end',
 'slice_start',
 'srow_x',
 'srow_y',
 'srow_z',
 'toffset',
 'vox_offset',
 'xyzt_units')

In [5]:
print('\\'.join(map(str, img.TransformIndexToPhysicalPoint((0, 0, 0)))))
print('\\'.join(map(str, img.TransformIndexToPhysicalPoint((0, 0, 1)))))
print('\\'.join(map(str, img.TransformIndexToPhysicalPoint((0, 0, 2)))))
print('\\'.join(map(str, img.TransformIndexToPhysicalPoint((0, 0, 3)))))

-190.283203125\-174.658203125\1274.5
-190.283203125\-174.658203125\1275.300000011921
-190.283203125\-174.658203125\1276.1000000238419
-190.283203125\-174.658203125\1276.9000000357628


## DCM序列文件基本信息
以分割软件导出的序列为例

In [6]:
# 序列中所有文件名
fnames = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(r"E:\other program\DCMProcessor\dataset\DCMExample")

# 声明，设置文件名
reader = sitk.ImageSeriesReader()
reader.SetFileNames(fnames)
# 开启对metadata的读取
reader.MetaDataDictionaryArrayUpdateOn()
reader.LoadPrivateTagsOn()

img = reader.Execute()

In [7]:
basic_info = (
    ("size\t", img.GetSize()),
    ("dimension", img.GetDimension()),
    ("origin\t", img.GetOrigin()),
    ("spacing\t", img.GetSpacing()),
    ("direction", img.GetDirection()),
    ("dtype\t", sitk.GetPixelIDValueAsString(img.GetPixelID())),
)
print("基本信息：")
for info in basic_info:
    print(f"{info[0]}\t:{info[1]}")

基本信息：
size		:(512, 512, 698)
dimension	:3
origin		:(-389.23828125, -628.73828125, -1244.3)
spacing		:(1.5234375, 1.5234375, 1.4)
direction	:(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
dtype		:16-bit signed integer


In [8]:
print("全部MetaData:")
for key in reader.GetMetaDataKeys(slice=0):
    values = []
    for slice in range(img.GetSize()[2]):
        values.append(reader.GetMetaData(slice, key))
    values_set = set(values)
    if len(values_set) == 1:
        print(f"{key} 全等,值为{reader.GetMetaData(0, key)}")
    else:
        print(f"{key} *************有{len(values_set)}个值")

全部MetaData:
0008|0005 全等,值为ISO_IR 192
0008|0008 全等,值为DERIVED\PRIMARY\AXIAL\CT_SOM5 SPI 
0008|0016 全等,值为1.2.840.10008.5.1.4.1.1.2
0008|0018 *************有698个值
0008|0020 全等,值为20190513
0008|0021 全等,值为20190513
0008|0030 全等,值为170434.535000 
0008|0031 全等,值为170611.353000 
0008|0050 全等,值为A10173736670
0008|0060 全等,值为CT
0008|0070 全等,值为SIEMENS 
0008|0080 全等,值为HUG 
0008|0081 全等,值为Rue Gabrielle-Perret-Gentil
Geneve
District
CH 
0008|0090 全等,值为HUG^SRV-ONCO OH-NOR 
0008|1030 全等,值为PET^PETCT_13_CBM_spc (Adult)
0008|103e 全等,值为AC  CT WB  2.0  HD_FoV
0008|1070 全等,值为Antonio FIGUEIRAL 
0008|1090 全等,值为Biograph128 
0010|0010 全等,值为CRISP^DOMINIC 
0010|0020 全等,值为97755225
0010|0030 全等,值为19680906
0010|0040 全等,值为M 
0018|0050 全等,值为1.4 
0018|0060 全等,值为120 
0018|1020 全等,值为VG75B 
0018|1030 全等,值为PETCT_13_CBM_spc
0018|5100 全等,值为FFS 
0020|000d 全等,值为2.16.840.1.113669.632.20.1211.10003883407
0020|000e 全等,值为1.3.12.2.1107.5.1.4.11008.30000019051306533151100015244
0020|0010 全等,值为A10173736629
0020|0011 全等,值为2 
0020|0012 全等,值为2

In [9]:
"""
print("SliceLocation:\t\t", reader.GetMetaData("0020|1041"))
print("ImagePositionPatient:\t", reader.GetMetaData("0020|0032"))
"""
print("关键相同MetaData：")
common_tags = [
    ("0008|0008", "ImageType"),
    ("0008|0021", "SeriesDate"),
    ("0008|0031", "SeriesTime"),
    ("0008|0060", "Modality"),
    ("0008|1030", "StudyDescription"),  # DeepViewer中显示
    ("0008|103e", "SeriesDescription"), # DeepViewer中显示
    ("0020|000e", "SeriesInstanceUID"),
    ("0020|0037", "ImageOrientationPatient"),

    ("0010|0010", "PatientName"),

    ("0028|0100", "BitsAllocated"),
    ("0028|0101", "BitsStored\t"),
    ("0028|0102", "HighBit\t"),
    ("0028|0103", "PixelRepresentation"), # 0:unsigned, 1:signed
]
for tag in common_tags:
    print(f"{tag[0]}, {tag[1]}:\t {reader.GetMetaData(slice=0, key=tag[0])}")

关键相同MetaData：
0008|0008, ImageType:	 DERIVED\PRIMARY\AXIAL\CT_SOM5 SPI 
0008|0021, SeriesDate:	 20190513
0008|0031, SeriesTime:	 170611.353000 
0008|0060, Modality:	 CT
0008|1030, StudyDescription:	 PET^PETCT_13_CBM_spc (Adult)
0008|103e, SeriesDescription:	 AC  CT WB  2.0  HD_FoV
0020|000e, SeriesInstanceUID:	 1.3.12.2.1107.5.1.4.11008.30000019051306533151100015244
0020|0037, ImageOrientationPatient:	 1\0\0\0\1\0 
0010|0010, PatientName:	 CRISP^DOMINIC 
0028|0100, BitsAllocated:	 16
0028|0101, BitsStored	:	 12
0028|0102, HighBit	:	 11
0028|0103, PixelRepresentation:	 0


In [10]:
print("关键不同MetaData:")
different_tags = [
    ("0020|0013", "InstanceNumber"),
    ("0020|0032", "ImagePositionPatient"),
]
for tag in different_tags:
    print(f"{tag[0]}, {tag[1]}:\t {reader.GetMetaData(slice=697, key=tag[0])}")

关键不同MetaData:
0020|0013, InstanceNumber:	 1 
0020|0032, ImagePositionPatient:	 -389.23828125\-628.73828125\-268.5


In [11]:
kwargs = {
    "1": 11,
    "2": 22,
}
print(kwargs.keys())

dict_keys(['1', '2'])


## 写文件

In [12]:
writer = sitk.ImageFileWriter()
writer.GetRegisteredImageIOs()

('BMPImageIO',
 'BioRadImageIO',
 'Bruker2dseqImageIO',
 'GDCMImageIO',
 'GE4ImageIO',
 'GE5ImageIO',
 'GiplImageIO',
 'HDF5ImageIO',
 'JPEGImageIO',
 'JPEG2000ImageIO',
 'LSMImageIO',
 'MINCImageIO',
 'MRCImageIO',
 'MetaImageIO',
 'NiftiImageIO',
 'NrrdImageIO',
 'PNGImageIO',
 'StimulateImageIO',
 'TIFFImageIO',
 'VTKImageIO')

## 测试
### 1. MetaData

In [13]:
fnames = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(r"E:\other program\DCMProcessor\dataset\ct_dcm")
uid = sitk.ImageSeriesReader.GetGDCMSeriesIDs(r"E:\other program\DCMProcessor\dataset\ct_dcm")
reader = sitk.ImageSeriesReader()
reader.SetFileNames(fnames)
# 开启对metadata的读取
reader.MetaDataDictionaryArrayUpdateOn()
reader.LoadPrivateTagsOn()

img = reader.Execute()

print("全部MetaData:")
for key in reader.GetMetaDataKeys(slice=0):
    values = []
    for slice in range(img.GetSize()[2]):
        values.append(reader.GetMetaData(slice, key))
    values_set = set(values)
    if len(values_set) == 1:
        print(f"{key} 全等,值为{reader.GetMetaData(0, key)}")
    else:
        print(f"{key} *************有{len(values_set)}个值")

全部MetaData:
0008|0016 全等,值为1.2.840.10008.5.1.4.1.1.2
0008|0018 *************有851个值
0008|0020 全等,值为20220427
0008|0021 全等,值为20220427
0008|0030 全等,值为103702
0008|0031 全等,值为103702
0008|0050 全等,值为
0008|0060 全等,值为CT
0008|0090 全等,值为
0010|0010 全等,值为test name 
0010|0020 全等,值为
0010|0030 全等,值为
0010|0040 全等,值为
0020|000d 全等,值为1.20220427.1103702
0020|000e 全等,值为1.2.826.0.1.3680043.2.1125.20220427.1103702
0020|0010 全等,值为
0020|0011 全等,值为
0020|0013 *************有851个值
0020|0032 *************有851个值
0020|0037 全等,值为1\0\0\0\1\0 
0028|0002 全等,值为1
0028|0004 全等,值为MONOCHROME2 
0028|0010 全等,值为512
0028|0011 全等,值为512
0028|0030 全等,值为.683000028133392\.683000028133392 
0028|0100 全等,值为16
0028|0101 全等,值为16
0028|0102 全等,值为15
0028|0103 全等,值为1
0028|1052 全等,值为0 
0028|1053 全等,值为1 
0028|1054 全等,值为US
ITK_original_direction 全等,值为[UNKNOWN_PRINT_CHARACTERISTICS]

ITK_original_spacing 全等,值为[UNKNOWN_PRINT_CHARACTERISTICS]



### 2. 图像位置、方向

In [14]:
print("导出的DCM：")
fnames = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(r"E:\other program\DCMProcessor\dataset\ct_dcm")
uid = sitk.ImageSeriesReader.GetGDCMSeriesIDs(r"E:\other program\DCMProcessor\dataset\ct_dcm")
reader = sitk.ImageSeriesReader()
reader.SetFileNames(fnames)
reader.MetaDataDictionaryArrayUpdateOn()
reader.LoadPrivateTagsOn()
img = reader.Execute()
basic_info = (
    ("size\t", img.GetSize()),
    ("dimension", img.GetDimension()),
    ("origin\t", img.GetOrigin()),
    ("spacing\t", img.GetSpacing()),
    ("direction", img.GetDirection()),
    ("dtype\t", sitk.GetPixelIDValueAsString(img.GetPixelID())),
)
for info in basic_info:
    print(f"{info[0]}\t:{info[1]}")

print("原始nii:")
reader = sitk.ImageFileReader()
reader.SetFileName(r"E:\other program\DCMProcessor\dataset\ct.nii")
img = reader.Execute()
basic_info = (
    ("size\t", img.GetSize()),
    ("dimension", img.GetDimension()),
    ("origin\t", img.GetOrigin()),
    ("spacing\t", img.GetSpacing()),
    ("direction", img.GetDirection()),
    ("dtype\t", sitk.GetPixelIDValueAsString(img.GetPixelID())),
)
for info in basic_info:
    print(f"{info[0]}\t:{info[1]}")

导出的DCM：
size		:(512, 512, 851)
dimension	:3
origin		:(-190.283203125, -174.658203125, 1274.5)
spacing		:(0.683000028133392, 0.683000028133392, 0.8000000119209295)
direction	:(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
dtype		:16-bit signed integer
原始nii:
size		:(512, 512, 851)
dimension	:3
origin		:(-190.283203125, -174.658203125, 1274.5)
spacing		:(0.6830000281333923, 0.6830000281333923, 0.800000011920929)
direction	:(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
dtype		:16-bit signed integer
